# XGBoost time serien forecosting

In [3]:
import warnings
# hide warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sktime.utils.plotting import plot_series, plot_lags, plot_correlations
from sktime.forecasting.base import ForecastingHorizon
from sktime.split import temporal_train_test_split
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
from statsmodels.tsa.deterministic import DeterministicProcess
from xgboost import XGBRegressor
import xgboost as xgb

from math import sqrt
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import h2o
from h2o.estimators import H2OTargetEncoderEstimator

import pickle

In [4]:
# Set Matplotlib defaults
#print(plt.style.available)
#plt.style.use("seaborn-v0_8-whitegrid'")

plt.rc(
    "figure",
    autolayout=True,
    figsize=(11, 4),
    titlesize=18,
    titleweight='bold',
)

plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)

plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)

In [5]:
holidays_url = '../data//holidays_events.csv'
oil_url = '../data//oil.csv'
stores_url = '../data/stores.csv'
test_url = '../data/test.csv'
train_url = '../data//train.csv'
transactions_url = '../data/transactions.csv'

In [6]:
# Load Data to data frame
df_holidays = pd.read_csv(holidays_url)
df_oil = pd.read_csv(oil_url)
df_stores = pd.read_csv(stores_url)
df_test = pd.read_csv(test_url)
df_train = pd.read_csv(train_url)
df_transactions = pd.read_csv(transactions_url)

In [7]:
# Convert 'date' to datetime
df_holidays['date'] = pd.to_datetime(df_holidays['date'])
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])
df_transactions['date'] = pd.to_datetime(df_transactions['date']) 

In [8]:
# Filling missing oil prices
full_date_range = pd.date_range(start=df_oil['date'].min(), end=df_oil['date'].max())
full_date_df = pd.DataFrame({'date': full_date_range})
df_oil = pd.merge(full_date_df, df_oil, on='date', how='left')
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].interpolate(method='linear')

## Feature engineering

### Static features

In [9]:
#merge with oil price
df_train_fe = pd.merge(left= df_train, right= df_oil, how= 'left', on= 'date')
#merge with store info
df_train_fe = pd.merge(left= df_train_fe, right= df_stores, how= 'left', on= 'store_nbr')

#merge with holiday data

# Step 1: Split df_holidays based on the locale type
national_holidays = df_holidays[df_holidays['locale'] == 'National']
regional_holidays = df_holidays[df_holidays['locale'] == 'Regional']
local_holidays = df_holidays[df_holidays['locale'] == 'Local']

# Step 2: Merge each type separately

# 2.1 National Holidays: Merge only on date
df_merged_national = pd.merge(df_train_fe, national_holidays, on='date', how='left')

# 2.2 Regional Holidays: Merge on date and state (locale_name corresponds to state in df_train_fe)
df_merged_regional = pd.merge(
    df_train_fe, 
    regional_holidays, 
    left_on=['date', 'state'], 
    right_on=['date', 'locale_name'], 
    how='left'
)

# 2.3 Local Holidays: Merge on date and city (locale_name corresponds to city in df_train_fe)
df_merged_local = pd.merge(
    df_train_fe, 
    local_holidays, 
    left_on=['date', 'city'], 
    right_on=['date', 'locale_name'], 
    how='left'
)

# Step 3: Combine the results

# Start with National holidays
df_combined = df_merged_national.copy()

# Add columns from Regional merge, without duplicating
df_combined = df_combined.combine_first(df_merged_regional)

# Add columns from Local merge, without duplicating
df_combined = df_combined.combine_first(df_merged_local)

# Clean-up step (optional): You can drop unnecessary columns or fill NaN values
# Drop locale_name columns if you want
df_combined = df_combined.drop(['locale_name'], axis=1, errors='ignore')

# Fill NaN values in the description or type columns as needed
df_combined['description'] = df_combined['description'].fillna('No Holiday')

#Create is_holiday column
df_combined['is_holiday'] = df_combined['type_y'].notna().astype(int)

# Show the combined dataframe
df_train_fe = df_combined.copy()

#rename columns
df_train_fe = df_train_fe.rename(columns= {'type_x': 'type_store',
                                           'cluster': 'cluster_store',
                                           'dcoilwtico': 'oil_price',
                                           'type_y': 'type_holiday',
                                           'locale': 'scale_holiday',
                                           'description': 'description_holiday',
                                           'transferred': 'transferred_holiday'
                                           })



In [10]:
df_train_fe.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type_store,cluster_store,type_holiday,scale_holiday,description_holiday,transferred_holiday,is_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,Quito,Pichincha,D,13,Holiday,National,Primer dia del ano,False,1
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN,Quito,Pichincha,D,13,Holiday,National,Primer dia del ano,False,1
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN,Quito,Pichincha,D,13,Holiday,National,Primer dia del ano,False,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN,Quito,Pichincha,D,13,Holiday,National,Primer dia del ano,False,1
4,4,2013-01-01,1,BOOKS,0.0,0,NaN,Quito,Pichincha,D,13,Holiday,National,Primer dia del ano,False,1


### Datetime features

In [11]:
# Extract useful features from the datetime column
df_train_fe['year'] = df_train_fe['date'].dt.year
df_train_fe['month'] = df_train_fe['date'].dt.month
df_train_fe['weekofyear'] = df_train_fe['date'].dt.isocalendar().week
df_train_fe['dayofyear'] = df_train_fe['date'].dt.dayofyear
df_train_fe['day'] = df_train_fe['date'].dt.day
df_train_fe['day_of_week'] = df_train_fe['date'].dt.dayofweek
df_train_fe['is_weekend'] = (df_train_fe['day_of_week'] >= 5).astype(int)
df_train_fe['quarter'] = df_train_fe['date'].dt.quarter


In [12]:
df_train_fe.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type_store,...,transferred_holiday,is_holiday,year,month,weekofyear,dayofyear,day,day_of_week,is_weekend,quarter
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,Quito,Pichincha,D,...,False,1,2013,1,1,1,1,1,0,1
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN,Quito,Pichincha,D,...,False,1,2013,1,1,1,1,1,0,1
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN,Quito,Pichincha,D,...,False,1,2013,1,1,1,1,1,0,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN,Quito,Pichincha,D,...,False,1,2013,1,1,1,1,1,0,1
4,4,2013-01-01,1,BOOKS,0.0,0,NaN,Quito,Pichincha,D,...,False,1,2013,1,1,1,1,1,0,1


### Lag features

In [13]:
def make_lags(ts, lags):

    """Function to create lag features

    Returns:
        df: df with added lag features
    """
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(1, lags + 1)
        },
        axis=1)

In [14]:
groups_train = df_train_fe.groupby(['store_nbr','family'])

df_combined = []

#feature engineering of data frames
for (store, family), df in groups_train:
    df['store_nbr'] = store
    df['family'] = family

    df_lags = make_lags(df['sales'], 39)
    df = pd.concat([df, df_lags], axis= 1)

    df_combined.append(df)


### Window features

In [15]:
# Create rolling features (moving averages)

for df in df_combined:

    df['rolling_7_sales'] = df['sales'].rolling(window=7).mean()
    df['rolling_14_sales'] = df['sales'].rolling(window=14).mean()
    df['rolling_30_sales'] = df['sales'].rolling(window=30).mean()

#Shift by one to avoid leakage
    df['rolling_7_sales'] = df['rolling_7_sales'].shift(1)
    df['rolling_14_sales'] = df['rolling_14_sales'].shift(1)
    df['rolling_30_sales'] = df['rolling_30_sales'].shift(1)


In [16]:
df_train_fe = pd.concat(df_combined, axis= 0)

### Drop unnecessary columns

In [17]:
df_train_fe.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion',
       'oil_price', 'city', 'state', 'type_store', 'cluster_store',
       'type_holiday', 'scale_holiday', 'description_holiday',
       'transferred_holiday', 'is_holiday', 'year', 'month', 'weekofyear',
       'dayofyear', 'day', 'day_of_week', 'is_weekend', 'quarter', 'y_lag_1',
       'y_lag_2', 'y_lag_3', 'y_lag_4', 'y_lag_5', 'y_lag_6', 'y_lag_7',
       'y_lag_8', 'y_lag_9', 'y_lag_10', 'y_lag_11', 'y_lag_12', 'y_lag_13',
       'y_lag_14', 'y_lag_15', 'y_lag_16', 'y_lag_17', 'y_lag_18', 'y_lag_19',
       'y_lag_20', 'y_lag_21', 'y_lag_22', 'y_lag_23', 'y_lag_24', 'y_lag_25',
       'y_lag_26', 'y_lag_27', 'y_lag_28', 'y_lag_29', 'y_lag_30', 'y_lag_31',
       'y_lag_32', 'y_lag_33', 'y_lag_34', 'y_lag_35', 'y_lag_36', 'y_lag_37',
       'y_lag_38', 'y_lag_39', 'rolling_7_sales', 'rolling_14_sales',
       'rolling_30_sales'],
      dtype='object')

In [18]:
columns_drop = ['city', 'state', 'type_holiday', 'scale_holiday', 'description_holiday', 'transferred_holiday']

df_train_fe.drop(columns= columns_drop, inplace= True)

### Drop NaN values

In [19]:
df_train_fe.isnull().sum()

id                      0
date                    0
store_nbr               0
family                  0
sales                   0
onpromotion             0
oil_price            1782
type_store              0
cluster_store           0
is_holiday              0
year                    0
month                   0
weekofyear              0
dayofyear               0
day                     0
day_of_week             0
is_weekend              0
quarter                 0
y_lag_1              1782
y_lag_2              3564
y_lag_3              5346
y_lag_4              7128
y_lag_5              8910
y_lag_6             10692
y_lag_7             12474
y_lag_8             14256
y_lag_9             16038
y_lag_10            17820
y_lag_11            19602
y_lag_12            21384
y_lag_13            23166
y_lag_14            24948
y_lag_15            26730
y_lag_16            28512
y_lag_17            30294
y_lag_18            32076
y_lag_19            33858
y_lag_20            35640
y_lag_21    

In [20]:
df_train_fe = df_train_fe.dropna()

In [21]:
df_train_fe = df_train_fe.sort_values('id')

### Categorical data

In [22]:
cat_columns = ['store_nbr', 'family', 'type_store', 'cluster_store']

# Convert columns to categorical
df_train_fe[cat_columns] = df_train_fe[cat_columns].apply(lambda col: col.astype('category'))

In [24]:
def encoding(df_train_fe, cat_columns, encoder):

    for cat in cat_columns:

        if encoder == 'label':

            le = LabelEncoder()

            for cat in cat_columns:
                df_train_fe[cat] = le.fit_transform(df_train_fe[cat])


        elif encoder == 'one':

            for cat in cat_columns:
                df_train_fe = pd.get_dummies(df_train_fe, columns=[cat])


        elif encoder == 'target':

            # Train a TE model
            encoder_te = H2OTargetEncoderEstimator(data_leakage_handling="leave_one_out",
                                                    blending=True,
                                                    inflection_point=3,
                                                    smoothing=10,
                                                    noise=0.15     # In general, the less data you have the more regularization you need
                                                )

            encoder_te.train(x= cat_columns,
                            y= df_train_fe['sales'],
                            training_frame= df_train_fe)

            # New target encoded train and test sets
            df_train_fe = encoder_te.transform(frame= df_train_fe, as_training=True) #True for training data, False for test data
                
    return df_train_fe




In [26]:
df_train_fe_test = encoding(df_train_fe, cat_columns, encoder= 'one')

KeyError: "None of [Index(['store_nbr'], dtype='object')] are in the [columns]"

In [104]:
df_train_fe

,id,date,store_nbr,family,sales,onpromotion,oil_price,type_store,cluster_store,is_holiday,...,y_lag_33,y_lag_34,y_lag_35,y_lag_36,y_lag_37,y_lag_38,y_lag_39,rolling_7_sales,rolling_14_sales,rolling_30_sales
69498,69498,2013-02-09,1,AUTOMOTIVE,4.000,0,96.143333,D,13,0,...,0.000,2.000,5.000,3.000,3.000,2.00000,0.000000,1.285714,2.071429,2.033333
69499,69499,2013-02-09,1,BABY CARE,0.000,0,96.143333,D,13,0,...,0.000,0.000,0.000,0.000,0.000,0.00000,0.000000,0.000000,0.000000,0.000000
69500,69500,2013-02-09,1,BEAUTY,4.000,0,96.143333,D,13,0,...,1.000,0.000,3.000,3.000,0.000,2.00000,0.000000,2.285714,1.785714,1.600000
69501,69501,2013-02-09,1,BEVERAGES,783.000,0,96.143333,D,13,0,...,1158.000,407.000,1160.000,953.000,919.000,1091.00000,0.000000,811.714286,830.642857,916.566667
69502,69502,2013-02-09,1,BOOKS,0.000,0,96.143333,D,13,0,...,0.000,0.000,0.000,0.000,0.000,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3008011,3000883,2017-08-15,9,POULTRY,438.133,0,47.570000,B,6,0,...,317.900,350.776,323.436,464.730,686.079,509.43402,439.091000,369.654711,447.313353,428.532999
3008012,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,47.570000,B,6,0,...,92.945,80.142,103.859,126.990,145.922,145.90399,108.145004,115.663141,113.432428,104.690466
3008013,3000885,2017-08-15,9,PRODUCE,2419.729,148,47.570000,B,6,0,...,1252.735,1388.688,2182.856,1603.893,2360.294,1884.58400,1227.892000,1508.710857,1609.729214,1593.872033
3008014,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.570000,B,6,0,...,5.000,1.000,3.000,2.000,2.000,1.00000,2.000000,151.571429,155.928571,77.700000


## XGBoost model